In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append('../')    
import GSSUtility as GU
import statsmodels.formula.api as smf
from pandas.rpy import common as com

/home/misha/python/pandas-0.16.0-py2.7-linux-x86_64.egg/pandas/rpy/__init__.py:8: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2, found here: http://rpy.sourceforge.net
  "like rpy2, found here: http://rpy.sourceforge.net", FutureWarning)


In [2]:
pathToData = '../../Data/'
dataCont = GU.dataContainer(pathToData)

Loading DataFrame df. This may take a few minutes.


In [3]:
def independent_columns(A, tol = 1e-05):
    """
    Return an array composed of independent columns of A.

    Note the answer may not be unique; this function returns one of many
    possible answers.

    http://stackoverflow.com/q/13312498/190597 (user1812712)
    http://math.stackexchange.com/a/199132/1140 (Gerry Myerson)
    http://mail.scipy.org/pipermail/numpy-discussion/2008-November/038705.html
        (Anne Archibald)

    >>> A = np.array([(2,4,1,3),(-1,-2,1,0),(0,0,2,2),(3,6,2,5)])
    >>> independent_columns(A)
    np.array([[1, 4],
              [2, 5],
              [3, 6]])
    """
    Q, R = np.linalg.qr(A.dropna())
    independent = np.where(np.abs(R.diagonal()) > tol)[0]
    print independent
    return A.iloc[:, independent]

In [4]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects
r = robjects.r
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import pandas.rpy.common as com
# import GSSUtility as GU
from rpy2.robjects.packages import importr
# R's "base" package
amelia = importr('Amelia')
mi = importr('mi')
mi = r['mi']
df = pd.read_csv('../../Data/test_collinear.csv', index_col=0)
# df.index = range(len(df))

In [5]:
articlesToUse = GU.filterArticles(dataCont.articleClasses, GSSYearsUsed=True, GSSYearsPossible=False, centralIVs=True)
article = [a for a in articlesToUse if a.articleID == 6759][0]

In [6]:
article.IVs, article.DVs, article.GSSYearsUsed, article.centralIVs, article.GSSYearsUsed

(['ATTEND', 'DEGREE', 'INCOME', 'RINCOME'],
 ['GRASS', 'HOMOSEX', 'PORNLAW'],
 [1976],
 ['ATTEND', 'DEGREE', 'INCOME', 'RINCOME'],
 [1976])

In [7]:
formula2 = '''
standardize(SATJOB, ddof=1) ~ JOBPROMO '''
DV = article.DVs[0]

In [12]:
design = dataCont.df.loc[1976, [DV] + article.IVs]
design.index = range(len(design))
design.head()

,GRASS,ATTEND,DEGREE,INCOME,RINCOME
0,2,7,1,NaN,NaN
1,1,1,1,12,NaN
2,2,3,3,12,12
3,2,1,1,10,9
4,1,3,4,11,11


In [23]:
mi(com.convert_to_r_dataframe(design))

Error in .local(object, ...) : unused argument (nimp = 2)


RRuntimeError: Error in .local(object, ...) : unused argument (nimp = 2)


In [30]:
summary = r['summary']

In [36]:
print r('summary(imp$imp)')

Length  Class   Mode 
     0   NULL   NULL 



In [58]:
print com.convert_robj(r['imp1']).DEGREE.value_counts()
print
print design.DEGREE.value_counts()

1    732
0    529
3    153
4     61
2     24
dtype: int64

1    728
0    528
3    152
4     61
2     24
dtype: int64


In [59]:
rcode='''
    library(mi)
    mydf = %s
    IMP = mi(mydf, n.imp=2, n.iter=6, max.minutes=1)
    imp1 <- mi.data.frame(IMP, m = 1)
''' % com.convert_to_r_dataframe(design).r_repr()
r(rcode)
com.convert_robj(r['imp1'])

Beginning Multiple Imputation ( Tue Apr 14 12:28:16 2015 ):
Iteration 1 
 Chain 1 : GRASS*  ATTEND*  DEGREE*  INCOME*  RINCOME*  
 Chain 2 : GRASS*  ATTEND*  DEGREE*  INCOME*  RINCOME*  
Iteration 2 
 Chain 1 : GRASS*  ATTEND*  DEGREE*  INCOME   RINCOME   
 Chain 2 : GRASS*  ATTEND   DEGREE*  INCOME*  RINCOME   
Iteration 3 
 Chain 1 : GRASS   ATTEND   DEGREE*  INCOME   RINCOME   
 Chain 2 : GRASS   ATTEND   DEGREE*  INCOME*  RINCOME   
Iteration 4 
 Chain 1 : GRASS*  ATTEND*  DEGREE   INCOME   RINCOME*  
 Chain 2 : GRASS   ATTEND   DEGREE   INCOME   RINCOME   
Iteration 5 
 Chain 1 : GRASS   ATTEND   DEGREE   INCOME   RINCOME   
 Chain 2 : GRASS*  ATTEND   DEGREE   INCOME   RINCOME   
Iteration 6 
 Chain 1 : GRASS   ATTEND*  DEGREE*  INCOME*  RINCOME   
 Chain 2 : GRASS*  ATTEND   DEGREE   INCOME   RINCOME   
Reached the maximum iteration, mi did not converge ( Tue Apr 14 12:28:21 2015 )
Run 20 more iterations to mitigate the influence of the noise...
Beginning Multiple Imputation ( T

,GRASS,ATTEND,DEGREE,INCOME,RINCOME
0,2,7.000000,1,6.093628,0.656369
1,1,1.000000,1,12.000000,8.756615
2,2,3.000000,3,12.000000,12.000000
3,2,1.000000,1,10.000000,9.000000
4,1,3.000000,4,11.000000,11.000000
5,2,4.000000,0,1.000000,6.143698
6,2,4.000000,1,7.000000,4.000000
7,2,5.000000,0,9.000000,8.000000
8,1,1.000000,0,9.000000,9.611503
9,1,1.000000,1,12.000000,8.000000


In [60]:
DV = 'GRASS'
for year in article.GSSYearsUsed:
    design = dataCont.df.loc[year, [DV] + article.IVs]
#     design = design.fillna(design.mean())
    formula = GU.createFormula(dataCont, design)
#     results = smf.ols(formula2, data=design.dropna()).fit()
    
    #impute
    try:
        design.iloc[:,:] = com.convert_robj(dataCont.complete(dataCont.mice(design.values, m=1))).values
        print 'imputing worked fine'
    except:
        print 'imputing didnt work'
        print year, DV, article.IVs
        nominals = GU.createFormula(dataCont, design, return_nominals=True)
        non_nominals = list(set(design.columns) - set(nominals)) # list because sets are unhashable and cant be used for indices
        if len(non_nominals)>0: 
            design[non_nominals] = design[non_nominals].fillna(design[non_nominals].mean()) # the naive way
        if len(nominals)>0:
            design[nominals] = design[nominals].fillna(design[nominals].mode())
            
    print smf.ols(formula, data=design.dropna()).fit().summary()
# 


 iter imp variable
  1   1  V1  V2  V3  V4  V5
  2   1  V1  V2  V3  V4  V5
  3   1  V1  V2  V3  V4  V5
  4   1  V1  V2  V3  V4  V5
  5   1  V1  V2  V3  V4  V5
imputing worked fine
                                OLS Regression Results                                
Dep. Variable:     standardize(GRASS, ddof=1)   R-squared:                       0.099
Model:                                    OLS   Adj. R-squared:                  0.096
Method:                         Least Squares   F-statistic:                     40.99
Date:                        Thu, 09 Apr 2015   Prob (F-statistic):           1.24e-32
Time:                                15:51:27   Log-Likelihood:                -2048.4
No. Observations:                        1499   AIC:                             4107.
Df Residuals:                            1494   BIC:                             4133.
Df Model:                                   4                                         
Covariance Type:                    

In [68]:
DV = 'GRASS'
for year in article.GSSYearsUsed:
    design = dataCont.df.loc[year, [DV] + article.IVs]
    design = design.fillna(design.mean())
    formula = GU.createFormula(dataCont, design)
#     results = smf.ols(formula2, data=design.dropna()).fit()
    
           
    print smf.ols(formula, data=design.dropna()).fit().summary()
# 

                                OLS Regression Results                                
Dep. Variable:     standardize(GRASS, ddof=1)   R-squared:                       0.101
Model:                                    OLS   Adj. R-squared:                  0.099
Method:                         Least Squares   F-statistic:                     42.02
Date:                        Thu, 09 Apr 2015   Prob (F-statistic):           1.98e-33
Time:                                15:55:15   Log-Likelihood:                -2046.6
No. Observations:                        1499   AIC:                             4103.
Df Residuals:                            1494   BIC:                             4130.
Df Model:                                   4                                         
Covariance Type:                    nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('../Data/test_collinear.csv', index_col=0)
from scipy.stats import pearsonr

In [5]:
df.head()

,HAPMAR,AGE,AGEWED,ATTEND,CHILDS,DIVORCE,EARNRS,EDUC,HOMPOP,INCOME,MARITAL,PRESTIGE,RACE,RINCOME,SEX,SPEDUC,SPWRKSTA,VOTE72,WRKSTAT,XNORCSIZ
0,1,56,23,7,1,2,2,12,2,NaN,1,46,1,NaN,2,12,1,1,2,3
1,2,51,20,1,1,2,1,12,2,12,1,45,1,NaN,2,12,1,2,7,3
2,2,41,23,3,3,2,3,16,5,12,1,69,1,12,1,12,2,1,1,3
3,1,55,21,1,2,2,2,12,5,10,1,44,1,9,1,12,7,1,1,3
4,1,32,24,3,3,2,1,18,5,11,1,60,1,11,1,12,7,1,1,3


array([ 1.])